In [1]:
import pandas as pd

In [2]:
business_df = pd.read_json('dataset/business.json',orient='records', lines=True)


In [3]:
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174567 entries, 0 to 174566
Data columns (total 15 columns):
address         174567 non-null object
attributes      174567 non-null object
business_id     174567 non-null object
categories      174567 non-null object
city            174567 non-null object
hours           174567 non-null object
is_open         174567 non-null int64
latitude        174566 non-null float64
longitude       174566 non-null float64
name            174567 non-null object
neighborhood    174567 non-null object
postal_code     174567 non-null object
review_count    174567 non-null int64
stars           174567 non-null float64
state           174567 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 20.0+ MB


In [4]:
business_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,"4855 E Warner Rd, Ste B9","{'AcceptsInsurance': True, 'ByAppointmentOnly'...",FYWN1wneV18bWNgQjJ2GNg,"[Dentists, General Dentistry, Health & Medical...",Ahwatukee,"{'Friday': '7:30-17:00', 'Tuesday': '7:30-17:0...",1,33.330690,-111.978599,Dental by Design,,85044,22,4.0,AZ
1,3101 Washington Rd,"{'BusinessParking': {'garage': False, 'street'...",He-G7vWjzVUysIKrfNbPUQ,"[Hair Stylists, Hair Salons, Men's Hair Salons...",McMurray,"{'Monday': '9:00-20:00', 'Tuesday': '9:00-20:0...",1,40.291685,-80.104900,Stephen Szabo Salon,,15317,11,3.0,PA
2,"6025 N 27th Ave, Ste 1",{},KQPW8lFf1y5BT2MxiSZ3QA,"[Departments of Motor Vehicles, Public Service...",Phoenix,{},1,33.524903,-112.115310,Western Motor Vehicle,,85017,18,1.5,AZ
3,"5000 Arizona Mills Cr, Ste 435","{'BusinessAcceptsCreditCards': True, 'Restaura...",8DShNS-LuFqpEWIp0HxijA,"[Sporting Goods, Shopping]",Tempe,"{'Monday': '10:00-21:00', 'Tuesday': '10:00-21...",0,33.383147,-111.964725,Sports Authority,,85282,9,3.0,AZ
4,581 Howe Ave,"{'Alcohol': 'full_bar', 'HasTV': True, 'NoiseL...",PfOCPjBrlQAnz__NXj9h_w,"[American (New), Nightlife, Bars, Sandwiches, ...",Cuyahoga Falls,"{'Monday': '11:00-1:00', 'Tuesday': '11:00-1:0...",1,41.119535,-81.475690,Brick House Tavern + Tap,,44221,116,3.5,OH


In [15]:
!python -m pip install missingno

In [16]:
import missingno as msno

In [17]:
msno.bar(business_df)

In [18]:
business_grouped_by_status = business_df[['is_open','stars']].groupby(by='is_open').sum().reset_index()
business_grouped_by_status.columns = ['open_status', 'count']
business_grouped_by_status.plot(kind='bar', x='open_status', y='count')


In [19]:
# eliminate restaurants that are closed
business_df = business_df[business_df['is_open'] == 1]

In [20]:
business_df.shape

(146702, 15)

In [36]:
most_business_info = business_df[['state', 'stars']].groupby(by='state').count().reset_index().sort_values(by='stars', ascending=False).head(5)
most_business_info.plot(kind='bar', x='state', y='stars')

AttributeError: 'AxesSubplot' object has no attribute 'show'

In [22]:
top_states = most_business_info['state']
business_df_top_states = business_df[business_df['state'].isin(top_states)]
business_df_top_states.shape

(118278, 15)

In [23]:
state_cat_dict = dict()
def check_row(row) :
    state = row['state']
    categories = row.categories
    if state in state_cat_dict :
        state_obj = state_cat_dict[state]
        for cat in categories :
            if cat in state_obj :
                #cat exist
                state_obj[cat] = state_obj[cat] + 1
            else :
                # cat does not exist in dict
                state_obj[cat] = 1
        state_cat_dict[state] = state_obj
    else :
        # new state
        state_obj = dict()
        for cat in categories :
            state_obj[cat] = 1
        state_cat_dict[state] = state_obj
        
temp = business_df_top_states.apply(lambda x : check_row(x), axis=1)


In [24]:
cat_dict = dict()
def check_row(row) :

    categories = row.categories
    for cat in categories :
        if cat in cat_dict :
            #cat exist
            cat_dict[cat] = cat_dict[cat] + 1
        else :
            # cat does not exist in dict
            cat_dict[cat] = 1
    
    
        
temp = business_df_top_states.apply(lambda x : check_row(x), axis=1)


In [25]:
cherry_picked_cat = ['Restaurants','Food','Nightlife','Bars','Fast Food','Sandwiches',
 'American (Traditional)','Pizza','Coffee & Tea','Burgers','Mexican','Breakfast & Brunch','Specialty Food',
 'Chinese','Italian','American (New)','Bakeries','Desserts','Chicken Wings','Salad','Beer','Wine & Spirits',
 'Pubs','Sushi Bars','Delis','Juice Bars & Smoothies','Steakhouses'
]
len(cherry_picked_cat)

27

In [26]:
state_cat_dict.keys()

dict_keys(['AZ', 'OH', 'NV', 'ON', 'NC'])

In [27]:
def check_if_restaurant(categories):
    for cat in categories :
        if cat in cherry_picked_cat :
            return True
    return False
    

In [28]:
business_df_top_states['restaurant'] = business_df_top_states.apply(lambda x : check_if_restaurant(x['categories']), axis = 1)


C:\Program Files\ADS\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
business_df_top_states = business_df_top_states[business_df_top_states['restaurant'] == True]

In [30]:
business_df_top_states.drop('restaurant', axis=1, inplace=True)

In [32]:
business_df_top_states.to_json('filtered_business.json', orient='records', lines=True)

In [33]:
filtered_business_df = pd.read_json('filtered_business.json',orient='records', lines=True)
filtered_business_df.shape

(39949, 15)